Book Reference: page 363-372 of **Python for Data Analysis Book by Wes McKinney**

# Chapter 12 Advanced Pandas

## Categorical Data

In [8]:
import numpy as np
import pandas as pd

In [9]:
values = pd.Series(['carrot', 'cabbage', 'carrot', 'carrot'] * 2)


In [10]:
values

0     carrot
1    cabbage
2     carrot
3     carrot
4     carrot
5    cabbage
6     carrot
7     carrot
dtype: object

In [11]:
pd.unique(values)

array(['carrot', 'cabbage'], dtype=object)

In [12]:
pd.value_counts(values)

carrot     6
cabbage    2
dtype: int64

Best practice in data warehousing is to use
- dimension tables 
    - contains the distinct values and storing the primary observations as integer keys referencing the dimension table

In [15]:
values = pd.Series([0, 1, 0, 0] * 2)
dim = pd.Series(['cabbage', 'carrot'])

In [16]:
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [17]:
dim

0    cabbage
1     carrot
dtype: object

We can use the take method to restore the original Series of strings:

In [18]:
dim.take(values)

0    cabbage
1     carrot
0    cabbage
0    cabbage
0    cabbage
1     carrot
0    cabbage
0    cabbage
dtype: object

The example above is called
**categorical or dictionary-encoded representation**

terms:
- categories - array of distinct values
- category codes - integer values referencing the categories

## Categorical Type in pandas

In [19]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2
N = len(fruits)

In [20]:
df = pd.DataFrame({'fruit': fruits,
                 'basket_id': np.arange(N),
                 'count': np.random.randint(3, 15, size=N),
                 'weight': np.random.uniform(0, 4, size=N)},
                 columns = ['basket_id', 'fruit', 'count', 'weight'])

In [21]:
df

,basket_id,fruit,count,weight
0,0,apple,12,3.812703
1,1,orange,12,0.189694
2,2,apple,4,3.354923
3,3,apple,9,1.077639
4,4,apple,9,1.585309
5,5,orange,13,2.874244
6,6,apple,5,3.420391
7,7,apple,5,2.877060


In [22]:
df['fruit']

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: object

In [23]:
fruit_cat = df['fruit'].astype('category')

In [24]:
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [25]:
c = fruit_cat.values

In [27]:
type(c)

pandas.core.arrays.categorical.Categorical

In [28]:
c.categories

Index(['apple', 'orange'], dtype='object')

In [29]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

In [30]:
df['fruit'] = df['fruit'].astype('category')

In [31]:
df.fruit

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

Given categories and codes, you can declare a Categorical object through

In [32]:
categories = ['a', 'b', 'c']
codes = [0, 1, 2, 0, 0, 1]

In [33]:
cat2 = pd.Categorical.from_codes(codes, categories)
cat2

[a, b, c, a, a, b]
Categories (3, object): [a, b, c]

You can make the categories ordered through adding a parameter `ordered = True` when declaring a Categorical object

In [34]:
ordered_cat = pd.Categorical.from_codes(codes, categories,
                                       ordered = True)

In [35]:
ordered_cat

[a, b, c, a, a, b]
Categories (3, object): [a < b < c]

In [36]:
cat2.as_ordered()

[a, b, c, a, a, b]
Categories (3, object): [a < b < c]

## Computations with Categoricals

In [37]:
np.random.seed(12345)
draws = np.random.randn(1000)

In [38]:
draws[:5]

array([-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057])

In [40]:
bins = pd.qcut(draws, 4) #quantile cut

In [41]:
bins

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.9499999999999997, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]]

In [42]:
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])

In [43]:
bins

[Q2, Q3, Q2, Q2, Q4, ..., Q3, Q2, Q1, Q3, Q4]
Length: 1000
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

In [45]:
bins.codes[:10]

array([1, 2, 1, 1, 3, 3, 2, 2, 3, 3], dtype=int8)

In [48]:
bins = pd.Series(bins, name='quartile')

In [49]:
results = (pd.Series(draws)
          .groupby(bins)
          .agg(['count', 'min', 'max'])
          .reset_index())

In [50]:
results

,quartile,count,min,max
0,Q1,250,-2.949343,-0.685484
1,Q2,250,-0.683066,-0.010115
2,Q3,250,-0.010032,0.628894
3,Q4,250,0.634238,3.927528


In [51]:
results['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

### Better performance with categoricals

In [53]:
N = 10000000 # 10 million
draws = pd.Series(np.random.randn(N))
labels = pd.Series(['a', 'b', 'c', 'd'] * (N // 4))

In [55]:
categories = labels.astype('category')

In [56]:
labels.memory_usage()

80000128

In [57]:
categories.memory_usage()

10000320

In [59]:
%time _ = labels.astype('category') 
# one time cost when converting series to categories

Wall time: 389 ms


### GroupBy operations can be significantly faster with categoricals

---

In [61]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)
cat_s = s.astype('category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]

In [62]:
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [63]:
actual_categories = ['a', 'b', 'c', 'd', 'e']
cat_s2 = cat_s.cat.set_categories(actual_categories)
# change categories using .set_categories func

In [64]:
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): [a, b, c, d, e]

Remove unused categories

In [65]:
cat_s3 = cat_s[cat_s.isin(['a', 'b'])]
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): [a, b, c, d]

In [66]:
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): [a, b]

---
### One-hot encoding
- Transforming categorical data into dummy variables